https://colab.research.google.com/github/corazzon/boostcourse-ds-511/blob/master/pima-classification-baseline-03.ipynb

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# %matplotlib inline

In [2]:
df = pd.read_csv('data/diabetes_feature.csv')
df.shape

(768, 16)

# 학습과 예측에 사용할 데이터셋 만들기

In [3]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [4]:
#feature
X = df[['Glucose', 'BloodPressure', 'SkinThickness',
    'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high', 
    'Insulin_nan', 'low_glu_insulin']]
X.shape

(768, 9)

In [1]:
#labels
y = df['Outcome']
y.shape



NameError: name 'df' is not defined

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

X_train.shape, y_train.shape

((614, 9), (614,))

In [7]:
X_test.shape, y_test.shape

((154, 9), (154,))

# 머신러닝 알고리즘 사용하기

In [8]:
# DecisionTree 를 불러옵니다.

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=11, random_state=42)
model

DecisionTreeClassifier(max_depth=11, random_state=42)

## 최적의 max_depth 값 찾기

In [9]:
for max_depth in range(3,12):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    y_predict = model.fit(X_train, y_train).predict(X_test)
    score = accuracy_score(y_test, y_predict)*100
    print(max_depth, score)

NameError: name 'accuracy_score' is not defined

## Grid Search
* https://scikit-learn.org/stable/modules/cross_validation.html
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [10]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier(random_state=42)

param_grid = {"max_depth":  range(3,12) ,
             "max_features":[0.3, 0.5, 0.7, 0.9, 1]}

clf = GridSearchCV(model, param_grid=param_grid, 
                   n_jobs=-1, #장비의 모든 프로세서 사용
                   cv=5,
                  verbose=1) #과정 로그가 찍혀나와
clf

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(3, 12),
                         'max_features': [0.3, 0.5, 0.7, 0.9, 1]},
             verbose=1)

In [11]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(3, 12),
                         'max_features': [0.3, 0.5, 0.7, 0.9, 1]},
             verbose=1)

In [12]:
clf.best_params_

{'max_depth': 5, 'max_features': 0.7}

In [13]:
clf.best_estimator_

DecisionTreeClassifier(max_depth=5, max_features=0.7, random_state=42)

In [14]:
clf.best_score_

0.8664934026389444

In [18]:
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
12,0.011188,0.004059,0.004745,0.001245,5,0.7,"{'max_depth': 5, 'max_features': 0.7}",0.878049,0.910569,0.813008,0.837398,0.893443,0.866493,0.036082,1
7,0.008998,0.000649,0.004096,0.000804,4,0.7,"{'max_depth': 4, 'max_features': 0.7}",0.813008,0.886179,0.829268,0.861789,0.918033,0.861655,0.037935,2
8,0.009832,0.000772,0.004314,0.000798,4,0.9,"{'max_depth': 4, 'max_features': 0.9}",0.821138,0.886179,0.853659,0.853659,0.893443,0.861615,0.026005,3
18,0.018735,0.013934,0.005386,0.001654,6,0.9,"{'max_depth': 6, 'max_features': 0.9}",0.829268,0.894309,0.821138,0.878049,0.877049,0.859963,0.029149,4
27,0.012678,0.002070,0.004137,0.001069,8,0.7,"{'max_depth': 8, 'max_features': 0.7}",0.861789,0.878049,0.837398,0.853659,0.860656,0.858310,0.013162,5


In [19]:
clf.predict(X_test)

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
      dtype=int64)

In [20]:
clf.score(X_test, y_test)

0.8701298701298701

## Random Search
<img src="https://i.imgur.com/N17CPGj.png">

출처 : http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf

In [23]:
model

DecisionTreeClassifier(random_state=42)

In [55]:
max_depth = np.random.randint(3, 20, 10) #10개의 값 
max_depth

array([ 7, 17, 11,  8, 12, 18,  6, 19, 19, 17])

In [56]:
max_features = np.random.uniform(0.7, 1.0, 100)

In [57]:
param_distributions = {'max_depth': max_depth, 
                           'max_features' : max_features,
                          "min_samples_split": list(range(2,7))
                          }
param_distributions

{'max_depth': array([ 7, 17, 11,  8, 12, 18,  6, 19, 19, 17]),
 'max_features': array([0.71548614, 0.94615294, 0.76487391, 0.71957994, 0.89971024,
        0.70547347, 0.89252343, 0.93416305, 0.95769277, 0.78480486,
        0.92455778, 0.74256084, 0.86129203, 0.79444705, 0.83823645,
        0.9791286 , 0.74583864, 0.81623622, 0.75844466, 0.84688794,
        0.94669838, 0.71700156, 0.75793569, 0.77044436, 0.7455558 ,
        0.92381024, 0.91214353, 0.88023283, 0.94369384, 0.83903936,
        0.89299414, 0.73322245, 0.95512502, 0.97157712, 0.9966317 ,
        0.91136503, 0.77957179, 0.98401465, 0.80750361, 0.71137127,
        0.76104069, 0.84966822, 0.9681092 , 0.86147037, 0.78868654,
        0.7005523 , 0.80671555, 0.83423636, 0.89508148, 0.91186693,
        0.89085158, 0.95996094, 0.70214814, 0.94527389, 0.78977707,
        0.96055565, 0.78715767, 0.92305108, 0.80077787, 0.94555291,
        0.8428967 , 0.94763923, 0.87703477, 0.85493823, 0.7925582 ,
        0.75483827, 0.73825187, 0.845

In [63]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomizedSearchCV(model,
        param_distributions,
        n_iter=1000, #10번을 돌면서 가장 좋은 하이퍼파라미터 찾겠다
        scoring= "accuracy",
        n_jobs=-1,
        cv=5,
        random_state=42    #값을 고정시켜주는 역할
                  ) #모든 자원 사용

clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'max_depth': array([ 7, 17, 11,  8, 12, 18,  6, 19, 19, 17]),
                                        'max_features': array([0.71548614, 0.94615294, 0.76487391, 0.71957994, 0.89971024,
       0.70547347, 0.89252343, 0.93416305, 0.95769277, 0.78480486,
       0.92455778, 0.74256084, 0.86129203, 0.79444705, 0.8382364...
       0.99984663, 0.71317061, 0.73532109, 0.71732752, 0.85889413,
       0.80367611, 0.96358197, 0.83286416, 0.8074239 , 0.7208739 ,
       0.94285765, 0.76525507, 0.72784349, 0.79099624, 0.75243386,
       0.9923513 , 0.72473384, 0.70759966, 0.99349873, 0.76301826,
       0.98197184, 0.7765909 , 0.89093882, 0.89719315, 0.85261779,
       0.84128754, 0.87818586, 0.91655905, 0.90175641, 0.90132004]),
                                        'min_samples_split': [2, 3, 4, 5, 6]},
                   random_state=42, scoring='accuracy')

In [64]:
clf.best_params_

{'min_samples_split': 4, 'max_features': 0.7652550686446491, 'max_depth': 8}

In [65]:
clf.best_score_

0.8697321071571371

In [66]:
clf.score(X_test, y_test)

0.8766233766233766

In [67]:
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
444,0.012371,0.004607,0.003922,0.000592,4,0.770444,8,"{'min_samples_split': 4, 'max_features': 0.770...",0.886179,0.853659,0.821138,0.902439,0.885246,0.869732,0.028985,1
963,0.012365,0.002381,0.004022,0.000545,4,0.742561,8,"{'min_samples_split': 4, 'max_features': 0.742...",0.886179,0.853659,0.821138,0.902439,0.885246,0.869732,0.028985,1
813,0.010295,0.001999,0.006262,0.001657,4,0.758445,8,"{'min_samples_split': 4, 'max_features': 0.758...",0.886179,0.853659,0.821138,0.902439,0.885246,0.869732,0.028985,1
772,0.010686,0.000732,0.003915,0.000145,4,0.738252,8,"{'min_samples_split': 4, 'max_features': 0.738...",0.886179,0.853659,0.821138,0.902439,0.885246,0.869732,0.028985,1
498,0.014120,0.001956,0.004381,0.001051,4,0.757936,8,"{'min_samples_split': 4, 'max_features': 0.757...",0.886179,0.853659,0.821138,0.902439,0.885246,0.869732,0.028985,1


# 학습과 예측하기

In [ ]:
model.fit(X_train, y_train)

In [ ]:
feature_names = X_train.columns.tolist()

In [ ]:
# from sklearn.tree import plot_tree

# plt.figure(figsize=(15, 15))
# tree = plot_tree(model, feature_names=feature_names, 
#                  fontsize=10, filled=True)

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(15, 15))
tree = plot_tree(model, 
                 feature_names=feature_names, 
                 fontsize=10, #글자 크기
                 filled=True) #색깔로 구분해서 보기

# 정확도(Accuracy) 측정하기

In [15]:
diff_count =abs(y_predict - y_test).sum()
diff_count

28

In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)*100

81.81818181818183